In [4]:
MTwobCpRB18N0lmIfAvoZC5g2aCbSLQcNfJjn122eJ3ed2wHEosziSC3 
# api key from pexels.com

NameError: name 'MTwobCpRB18N0lmIfAvoZC5g2aCbSLQcNfJjn122eJ3ed2wHEosziSC3' is not defined

In [8]:
import os
import requests
import logging
from moviepy.editor import VideoFileClip, concatenate_videoclips, TextClip, CompositeVideoClip, AudioFileClip
from transformers import pipeline
from ChatTTS import Chat
import torchaudio
import torch

# Set up logging
logging.basicConfig(level=logging.INFO)

# Hugging Face pipeline for text generation with GPU support
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available
text_generator = pipeline("text-generation", model="gpt2", device=device)

# Initialize ChatTTS for TTS
chat = Chat()

# Directory to save videos
os.makedirs("stock_videos", exist_ok=True)

# Pexels API key
PEXELS_API_KEY = "MTwobCpRB18N0lmIfAvoZC5g2aCbSLQcNfJjn122eJ3ed2wHEosziSC3"  # Use environment variable for API key

# Path to background music file
BACKGROUND_MUSIC_PATH = "path_to_your_background_music.mp3"  # Replace with your music file path

# Function to search and download a video from Pexels based on a query
def download_stock_video(prompt, index, duration=10):
    headers = {"Authorization": PEXELS_API_KEY}
    params = {"query": prompt, "per_page": 1}

    try:
        response = requests.get("https://api.pexels.com/videos/search", headers=headers, params=params)
        response.raise_for_status()
        videos = response.json().get("videos", [])

        if not videos:
            logging.warning(f"No videos found for query: {prompt}")
            return None

        video_url = videos[0]["video_files"][0]["link"]
        video_path = f"stock_videos/{prompt.replace(' ', '_')}_{index}.mp4"

        # Download the video
        with requests.get(video_url, stream=True) as r:
            r.raise_for_status()
            with open(video_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)

        # Trim video to required duration
        clip = VideoFileClip(video_path).subclip(0, duration)
        trimmed_video_path = f"{video_path}_trimmed.mp4"
        clip.write_videofile(trimmed_video_path, codec="libx264")
        return trimmed_video_path
    except requests.exceptions.RequestException as e:
        logging.error(f"Error downloading video: {e}")
        return None

# Generate subtitles for the video based on a prompt
def generate_subtitle(prompt):
    try:
        subtitle_text = text_generator(prompt, max_length=50, truncation=True)[0]["generated_text"]
        logging.info(f"Generated subtitle: {subtitle_text}")
        return subtitle_text
    except Exception as e:
        logging.error(f"Error generating subtitle: {e}")
        return ""

# Generate TTS audio from the subtitle text using ChatTTS
def generate_audio_with_chatt(texts, audio_path="output.wav"):
    try:
        wavs = chat.infer(texts)
        torchaudio.save(audio_path, torch.from_numpy(wavs[0]), 24000)
        return audio_path
    except Exception as e:
        logging.error(f"Error generating audio: {e}")
        return ""

# Combine video, subtitle, and audio
def create_video_with_subtitles(video_path, subtitle_text, audio_path, output_path="output_video.mp4"):
    try:
        clip = VideoFileClip(video_path)
        audio_clip = AudioFileClip(audio_path)

        # Load background music and set its volume
        background_music = AudioFileClip(BACKGROUND_MUSIC_PATH).volumex(0.1)  # Adjust volume as needed
        audio = CompositeAudioClip([audio_clip, background_music.set_duration(clip.duration)])
        clip = clip.set_audio(audio)

        # Create subtitle text clip
        txt_clip = (TextClip(subtitle_text, fontsize=24, color='white')
                    .set_position(("center", "bottom"))
                    .set_duration(clip.duration))

        # Overlay subtitles on video
        video = CompositeVideoClip([clip, txt_clip])
        video.write_videofile(output_path, codec="libx264")
        return output_path
    except Exception as e:
        logging.error(f"Error creating video with subtitles: {e}")
        return ""

# Main function to create a sequence of videos
def create_video_sequence(prompts, duration=10):
    final_clips = []
    for i, prompt in enumerate(prompts):
        logging.info(f"Processing prompt {i + 1}: {prompt}")

        video_path = download_stock_video(prompt, i + 1, duration)
        if not video_path:
            continue  # Skip if no video was found

        subtitle_text = generate_subtitle(prompt)
        if not subtitle_text:
            continue  # Skip if subtitle generation failed

        audio_path = generate_audio_with_chatt([subtitle_text])
        if not audio_path:
            continue  # Skip if audio generation failed

        video_with_subtitles_path = f"video_with_subtitles_{i + 1}.mp4"
        final_clip_path = create_video_with_subtitles(video_path, subtitle_text, audio_path, video_with_subtitles_path)

        if final_clip_path:
            final_clips.append(final_clip_path)

    # Concatenate all video clips
    if final_clips:
        video_clips = [VideoFileClip(path) for path in final_clips]
        final_video = concatenate_videoclips(video_clips, method="compose")
        final_video.write_videofile("final_output_video.mp4", codec="libx264")
    else:
        logging.warning("No clips to concatenate.")

# Example usage
prompts = [
    "A serene beach at sunrise",
    "A busy city street at night",
    "A peaceful mountain landscape"
]
create_video_sequence(prompts, duration=10)


INFO:root:Processing prompt 1: A serene beach at sunrise


Moviepy - Building video stock_videos/A_serene_beach_at_sunrise_1.mp4_trimmed.mp4.
Moviepy - Writing video stock_videos/A_serene_beach_at_sunrise_1.mp4_trimmed.mp4



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Moviepy - Done !
Moviepy - video ready stock_videos/A_serene_beach_at_sunrise_1.mp4_trimmed.mp4


INFO:root:Generated subtitle: A serene beach at sunrise on July 11, 2008. (Bastien Inzaurralde/The Washington Post)

A team investigating one of the worst cases of missing and murdered Americans on record is looking for what it hopes will
ERROR:root:Error generating audio: 
INFO:root:Processing prompt 2: A busy city street at night


...

INFO:root:Generated subtitle: A busy city street at night. After a few hours of being shut down for police brutality by angry protesters, a woman stood on top of the building to face police officers. A young man from a nearby residential area was also arrested by police, and
ERROR:root:Error generating audio: 
INFO:root:Processing prompt 3: A peaceful mountain landscape


Moviepy - Building video stock_videos/A_peaceful_mountain_landscape_3.mp4_trimmed.mp4.
Moviepy - Writing video stock_videos/A_peaceful_mountain_landscape_3.mp4_trimmed.mp4



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Moviepy - Done !
Moviepy - video ready stock_videos/A_peaceful_mountain_landscape_3.mp4_trimmed.mp4


INFO:root:Generated subtitle: A peaceful mountain landscape offers a perfect solution for a large town. As these hills rise high in the evening, it is a perfect choice for many people of all sizes (small to big, large to large). The hills are high in elevation and are
ERROR:root:Error generating audio: 


In [6]:
! pip install python-dotenv


In [15]:
! streamlit run main.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.12.226.255:8501

2024-12-07 21:27:37.303 Examining the path of torch.classes raised: Tried to instantiate class '__path__._path', but it does not exist! Ensure that it is registered via torch::class_
2024-12-07 21:28:15.433 Examining the path of torch.classes raised: Tried to instantiate class '__path__._path', but it does not exist! Ensure that it is registered via torch::class_
Moviepy - Building video project_assets/session_20241207_211602/story_video.mp4.
Moviepy - Writing video project_assets/session_20241207_211602/story_video.mp4

Moviepy - Done !                                                                
Moviepy - video ready project_assets/session_20241207_211602/story_video.mp4
2024-12-07 21:28:32.336 Examining the path of torch.classes raised: Tried to instantiate class '__path__._path', but it does not exist! Ensure that it is registered via torch::class

In [14]:
! pip install moviepy==1.0.3 numpy>=1.18.1 imageio>=2.5.0 decorator>=4.3.0 tqdm>=4.0.0 Pillow>=7.0.0 scipy>=1.3.0 pydub>=0.23.0 audiofile>=0.0.0 

In [2]:
! pip install manim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 4.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.5/661.5 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.2/585.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 2.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 945.1/945.1 kB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 2.4 MB/s eta 0:00:0000:0100:01
  Created wheel for manimpango: filename=M

In [ ]:
! streamlit run main.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.12.226.255:8501

2024-12-08 17:46:18.264 Examining the path of torch.classes raised: Tried to instantiate class '__path__._path', but it does not exist! Ensure that it is registered via torch::class_
2024-12-08 17:46:38.580 Uncaught app exception
Traceback (most recent call last):
  File "/home/pennydums/.pyenv/versions/3.10.15/envs/venv_torch_001/lib/python3.10/site-packages/streamlit/runtime/scriptrunner/exec_code.py", line 88, in exec_func_with_error_handling
    result = func()
  File "/home/pennydums/.pyenv/versions/3.10.15/envs/venv_torch_001/lib/python3.10/site-packages/streamlit/runtime/scriptrunner/script_runner.py", line 579, in code_to_exec
    exec(code, module.__dict__)
  File "/mnt/ssd_internal_victus/work/content_creator/main.py", line 345, in <module>
    audio_file_path = download_audio(video_id)
  File "/mnt/ssd_internal_victus/work/content_creator/main

In [3]:
! pip install ytmusicapi
! pip install pytube

In [6]:
! pip uninstall pytube

Found existing installation: pytube 15.0.0
Uninstalling pytube-15.0.0:
  Would remove:
    /home/pennydums/.pyenv/versions/3.10.15/envs/venv_torch_001/bin/pytube
    /home/pennydums/.pyenv/versions/3.10.15/envs/venv_torch_001/lib/python3.10/site-packages/pytube-15.0.0.dist-info/*
    /home/pennydums/.pyenv/versions/3.10.15/envs/venv_torch_001/lib/python3.10/site-packages/pytube/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
